# Using Evidently AI with a MinIO S3 Workspace

This notebook is a complete guide to setting up and running the Evidently UI service with a remote S3-compatible workspace (AWS S3, Google Cloud Storage, MinIO, etc.). We will use **MinIO** as our self-hosted object storage solution, which is fully compatible with the S3 API.

By the end of this tutorial, you will have:
1.  A running MinIO instance managed by Docker.
2.  A custom Evidently Docker image with the necessary S3 libraries.
3.  The Evidently UI service running and connected to your MinIO workspace.

This guide is structured into four parts:
- **Part 1:** Setting up MinIO with Docker Compose.
- **Part 2:** Building a Custom Evidently Docker Image with S3 Support.
- **Part 3:** Running the Evidently Service.
- **Bonus Tip:** Enhancing Security with an Unprivileged User.

> This tutorial also works if you're using Google Cloud Storage directly, just ignore the MinIO parts.

--- 
## Part 1: Setting Up MinIO with Docker Compose

First, we need an S3-compatible object storage server. MinIO is a perfect choice for local development. The easiest way to run it is with `docker-compose`, which allows us to define and manage the service cleanly.

> **Skippable Step:** If you already have an S3 bucket (on AWS S3, GCS, etc.), you can skip this part and go to Part 2 directly. Just make sure you have your Access Key, Secret Key, and the correct Endpoint URL ready.

#### **Step 1.1: Create a `docker-compose.yml` file**
Create a new file named `docker-compose.yml` and paste the following content into it. This configuration will start a MinIO server.

```yaml
# docker-compose.yml
version: '3.8'

services:
  minio:
    image: minio/minio:latest
    container_name: minio
    command: server /data --console-address ":9001"
    ports:
      - "9000:9000" # API Port
      - "9001:9001" # Console Port
    environment:
      - MINIO_ROOT_USER=minioadmin
      - MINIO_ROOT_PASSWORD=minioadmin
    volumes:
      - minio_data:/data

volumes:
  minio_data:
    driver: local
```

#### **Step 1.2: Start the MinIO Service**

Open a terminal in the same directory where you saved the `docker-compose.yml` file and run:

```bash
docker compose up -d
```

This command will download the MinIO image and start the service in the background.

> When you're finished with this tutorial, you can stop and remove the MinIO instance with the following command (ran from the same directory): `docker compose down`

#### **Step 1.3: Create a Bucket**

Now that MinIO is running, you need to create a "bucket" to store your Evidently projects.

1.  Open your web browser and navigate to **`http://localhost:9001`**.
2.  Log in with the credentials you set in the `docker-compose.yml` file:
    - **Username:** `minioadmin`
    - **Password:** `minioadmin`
3.  Click the **Create Bucket** button on the left sidebar.
4.  Name your bucket `evidently-ai` and click **Create Bucket**.

#### **Step 1.4 (Optional but Recommended): Create a Dedicated Access Key**

For better security, it's best to avoid using the root user (`minioadmin`) for applications. Let's create a new, dedicated user with its own Access Key and Secret Key.

> You can skip this step for dev purposes and use the `minioadmin` username and password as Access Key and Secret Key later.

1.  **Enter the MinIO container's shell:**
    ```bash
    docker exec -it minio /bin/sh
    ```
2.  **Configure the MinIO Client (`mc`) alias inside the container:**
    ```sh
    # Format: mc alias set <ALIAS_NAME> <SERVER_URL> <ROOT_USER> <ROOT_PASSWORD>
    mc alias set local http://localhost:9000 minioadmin minioadmin
    ```
3.  **Create the new user (this defines your new Access Key and Secret Key):**
    ```sh
    # Format: mc admin user add <ALIAS> <NEW_ACCESS_KEY> <NEW_SECRET_KEY>
    mc admin user add local evidently-user a-very-strong-secret-key
    ```
4.  **Assign permissions to the new user:**
    ```sh
    # Format: mc admin policy attach <ALIAS> <POLICY> --user <USER_ACCESS_KEY>
    mc admin policy attach local readwrite --user evidently-user
    ```
5.  **Exit the container:**
    ```sh
    exit
    ```
Now you have a new set of credentials! **Remember to use these in Part 3 instead of the `minioadmin` user.**

--- 
## Part 2: Building a Custom Evidently Docker Image

The official `evidently/evidently-service` image is minimal and does not include the `s3fs` or `gcsfs` library needed to communicate with remote data storage backends. We need to build a custom image that adds this dependency.

Here are two ways to do it.

### Method A: Build on Top of the Official Image (Recommended for Simplicity)
This is the easiest method. We create a new `Dockerfile` that uses the official image as a base and simply installs the required packages.

#### **Step 2.1.A: Create a Dockerfile**
Create a file named `Dockerfile.s3` and add the following content:

```Dockerfile
FROM evidently/evidently-service:latest

# Install the necessary libraries for S3 (s3fs) and GCS (gcsfs) support.
# You can remove the one you don't need.
RUN pip install s3fs gcsfs
```

#### **Step 2.2.A: Build the Image**
In your terminal, run the following command to build your custom image. We'll tag it as `evidently/evidently-service:s3`.

```bash
docker build -t evidently/evidently-service:s3 -f Dockerfile.s3 .
```

### Method B: Build from the Evidently Source Code
This method is for users who want to build directly from the official Evidently GitHub repository.

#### **Step 2.1.B: Clone the Repository**
```bash
git clone [https://github.com/evidentlyai/evidently.git](https://github.com/evidentlyai/evidently.git)
cd evidently
```

#### **Step 2.2.B: Build the Image with a Build Argument**
You can build the image by passing the `[fsspec]` extra to install all filesystem dependencies. It's also possible to install just `[s3]` or `[gcs]`.

```bash
# This command should be run from the root of the cloned 'evidently' repository
docker build \
  --build-arg INSTALL_EXTRAS="[fsspec]" \
  -f docker/Dockerfile.service \
  -t evidently/evidently-service:s3 .
```

--- 
## Part 3: Running the Custom Evidently Service

Now we have our MinIO instance running and our custom Docker image ready. Let's run the Evidently container and connect everything.

The command below will:
- Run our custom image (`evidently/evidently-service:s3`).
- Pass the MinIO credentials and endpoint URL as environment variables.
- Tell the Evidently UI to use our MinIO bucket as its workspace.

> For GCS, the workspace should be `gs://evidently-ai/workspace`, *assuming your bucket is also named **evidently-ai***.

Execute the following command in your terminal. **Remember to use the dedicated credentials you created in Step 1.4 if you completed that step.** If you didn't, don't worry! You can use the "minioadmin".

In [ ]:
docker run -d -p 8000:8000 \
  -e FSSPEC_S3_KEY="minioadmin" \
  -e FSSPEC_S3_SECRET="minioadmin" \
  -e FSSPEC_S3_ENDPOINT_URL="http://host.docker.internal:9000" \
  --name evidently-ui \
  evidently/evidently-service:s3 --workspace s3://evidently-ai/workspace

That works because the default `ENTRYPOINT` already starts the Evidently UI, we're just appending the workspace to it.

> **Important Note on `ENDPOINT_URL`:**
> - **Never** include a trailing slash (`/`) in the endpoint URL.

#### **Verify the Setup**

Open your browser and go to [http://localhost:8000](http://localhost:8000). You should now see the Evidently UI. You can create a new project, and it will be saved directly into your MinIO bucket! Check it from the [MinIO Console](http://localhost:9001).

--- 
## Bonus Tip: Enhancing Security with an Unprivileged User

Running Docker containers as the `root` user is a security risk. It's a best practice to create and switch to a non-root user inside your image. Here’s how you can modify the simple Dockerfile from **Method A** to add an unprivileged user.

#### **Create a `Dockerfile.s3.secure` file:**

```Dockerfile
FROM evidently/evidently-service:latest

# `s3fs` for S3-compatible storages
# `gcsfs` if you're using Google Cloud Storage  
RUN pip install s3fs gcsfs

# The `adduser` command is a user-friendly utility available in Debian-based images.
RUN adduser --disabled-password --uid 1000 evidentlyuser

USER evidentlyuser
```

You would then build and run this image just as before. This simple change significantly improves the security posture of your container.